In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git" -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:

In [3]:
!pip install "git+https://github.com/huggingface/transformers.git" -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
!pip install -U datasets -q

!pip install trl -q

In [28]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import torch
import random
import numpy as np
from datasets import Dataset, DatasetDict, ClassLabel, Sequence, load_metric

from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

from transformers import DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
from unsloth import FastLanguageModel
from trl import SFTTrainer

In [29]:
dataset = pd.read_csv('/content/drive/MyDrive/NER_CS/bida_version_2_and_prev_filtered_prompted.csv')

In [30]:
dataset.head()

,text,label,summary
0,1. Please explain how will you increase your p...,Shortfall,Due to system limitations and missing document...
1,Please upload the encashment certificate for t...,Shortfall,Numerous application shortcomings were identif...
2,Please submit last arrival seal Please submit ...,Shortfall,Critical documents are missing in the applicat...
3,1. Please upload the previous work permit canc...,Shortfall,Incomplete application due to missing document...
4,House rent is only taka 25000 which needs to b...,Shortfall,"Several issues hinder the application process,..."


In [31]:
dataset.drop("label", axis = 1, inplace=True)
dataset.head()

,text,summary
0,1. Please explain how will you increase your p...,Due to system limitations and missing document...
1,Please upload the encashment certificate for t...,Numerous application shortcomings were identif...
2,Please submit last arrival seal Please submit ...,Critical documents are missing in the applicat...
3,1. Please upload the previous work permit canc...,Incomplete application due to missing document...
4,House rent is only taka 25000 which needs to b...,"Several issues hinder the application process,..."


In [32]:
 # Split the data into training, validation, and test sets
train, X_temp = train_test_split(dataset, test_size=0.3, random_state=42)
validation, test = train_test_split(X_temp, test_size=0.10, random_state=42)

In [33]:
train = train.reset_index(drop=True)
validation = validation.reset_index(drop=True)
test = test.reset_index(drop=True)

In [34]:
 # Create a datasets.Dataset object
train = Dataset.from_dict(train)
validation = Dataset.from_dict(validation)
test = Dataset.from_dict(test)
# Print the dataset to check the format
print("Train Dataset : ",train)
print("Validation Dataset : ", validation)
print("Test Dataset : ", test)

Train Dataset :  Dataset({
    features: ['text', 'summary'],
    num_rows: 3924
})
Validation Dataset :  Dataset({
    features: ['text', 'summary'],
    num_rows: 1514
})
Test Dataset :  Dataset({
    features: ['text', 'summary'],
    num_rows: 169
})


In [35]:
# Create DatasetDict
dataset = DatasetDict({
    'validation': validation,
    'test': test,
    'train': train
})
print(dataset)

DatasetDict({
    validation: Dataset({
        features: ['text', 'summary'],
        num_rows: 1514
    })
    test: Dataset({
        features: ['text', 'summary'],
        num_rows: 169
    })
    train: Dataset({
        features: ['text', 'summary'],
        num_rows: 3924
    })
})


In [36]:
max_seq_length = 1024
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # True to use 4bit quantization / reduce memory usage (for T4 GPU)

In [37]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "mistralai/Mistral-7B-v0.1", #"unsloth/mistral-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

Unsloth: You passed in `mistralai/Mistral-7B-v0.1` and `load_in_4bit = True`.
We shall load `unsloth/mistral-7b-bnb-4bit` for 4x faster loading.


==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


### Add LoRa adapters

In [38]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # can improve fine-tuning, at attention/feed fwd layers
    target_modules = [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha = 16, # more change to pre-train weights but care overfitting
    lora_dropout = 0.05, # any, but 0 if perf opti.
    bias = "none",    # any, but "none" is perf  opti.
    use_gradient_checkpointing = True,
    random_state = 11,
    use_rslora = False,  # support rank stabilized LoRA
    loftq_config = None, # LoftQ support
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


### Data Preparation (Alpaca template)

In [39]:
prompt = """you are expert in text summarization. I have a text data of an applications.
your task is generate summary of the given text in one paragraph not list and less than 20 words:
### Text:
{}
### Summary:
{}"""

In [40]:
# Add EOS special token, according to previously loaded tokenizer
EOS_TOKEN = tokenizer.eos_token
def format_prompts(examples):
    inputs = examples["text"]
    outputs = examples["summary"]
    texts = []
    for inp, output in zip(inputs, outputs):
        # Add end of string token to prevent infinite generations.
        text = prompt.format(inp, output) + EOS_TOKEN
        texts.append(text)
    return {"text":texts,}

In [41]:
# Build prompts
dataset['train'] = dataset['train'].map(format_prompts, batched = True,)

# Print a sample :
dataset['train'][3]['text']

Map:   0%|          | 0/3924 [00:00<?, ? examples/s]

'you are expert in text summarization. I have a text data of an applications. \nyour task is generate summary of the given text in one paragraph not list and less than 20 words:\n### Text:\n1.Manpower table should fulfill properly 1.Ratio of Local and Foreign employee should be maintained A. Please see the BIDA guideline 2023 at the following link https bida.gov.bd storage app media pdf Guideline2023.pdf B. Please upload the document for the cancellation of the work permit as per the BIDA guideline page 28 10 upload the necessary document as per the BIDA guideline page 28 10 C. Please upload the passport copy each of the pages chronologically marked with the latest E visa seals exit visa seal latest arrival seal and final departure seal under the work permit D. Please upload the up to date income tax clearance certificate under section 91 of the Income Tax Ordinance 1984 up to the last departure date E. Please also upload the income tax certificate for the last two years for the expatr

In [42]:
dataset['validation'] = dataset['validation'].map(format_prompts, batched = True,)

Map:   0%|          | 0/1514 [00:00<?, ? examples/s]

### Train the model

In [49]:
training_args = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 8,
    warmup_steps = 5,
    num_train_epochs = 2,
    # max_steps = 110,
    learning_rate = 2e-4, # 2e-5
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 11,
    output_dir = "/content/drive/MyDrive/NER_CS/Mistral_BIDA_Text_Sum",
    run_name = "mistral-7b-base_en_v11",
    logging_strategy = 'steps',
    logging_steps = 256,
    evaluation_strategy='steps',
    save_strategy="steps",
    save_steps=98,
    save_total_limit = 2,
    )

In [50]:
# init trainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    eval_dataset = dataset['validation'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2, # if packing = False, else default to None
    packing = False, # more speed if packing short sequences. Maybe later
    args = training_args,
)

Map (num_proc=2):   0%|          | 0/3924 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1514 [00:00<?, ? examples/s]

In [ ]:
# train
trainer = trainer.train()

### Inference

Run the model we just trained. With Unsloth `FastLanguageModel`, test on one sample.

In [ ]:
print(dataset['test'][22])
comment = dataset['test'][22]['tokens']
print(comment)

#### One sample

In [17]:
# load LoRa adapters from HuggingFace
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/MyDrive/NER_CS/Mistral_BIDA/checkpoint-10",
    max_seq_length = 2024,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

FastLanguageModel.for_inference(model)

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Unsloth 2024.3 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [26]:
import re
inputs = tokenizer(
    [
        prompt.format(
        dataset['test']['text'][8], # insert comment
        "", # output - blank (instead of label) for generation
        )
    ],
    return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 250, use_cache = True)
decoded_outputs = tokenizer.batch_decode(outputs)

decoded_outputs = str(decoded_outputs[0])
print(decoded_outputs)
# # Using regular expression to extract the label
# label_match = re.search(r'### label:\n(\d+)', decoded_outputs)

# if label_match:
#     label = label_match.group(1)
#     print("Extracted label:", label)
# else:
#     print("Label not found.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> you are expert in text summarization. I have a text data of an applications. 
your task is generate summary of the given text in one paragraph not list and less than 20 words:
### Text:
1.Registration letter of BIDA should be submitted 2.Basic salary should not be less than USD 850 or equivalent 3.Designation mention in application form and appointment letter does not match with advertisement 1. Please upload a the copy of the passport mark with the last departure seal b Encashment certificate of inward remittance of minium U.S 50 000.00 as the initial investment of this company. travel history is not correct. 2. did not upload the copy of the passport whole used part . Please explain provide the followings 1. The agreement does not have any name and designation of the signatories. 2. Proof e.g. board resolution authorization letter from both ends of the people being authorized to sign the agreement is required. 3. Since the agreement is only in a white page please notarize it and 

In [ ]:
Applications are incomplete due to missing registration letters, incorrect basic salaries, mismatched designations, and incomplete passport information. Other issues include incorrect travel history, missing encashment certificates, and incomplete agreements.</s>

In [24]:
dataset['test']['summary'][8]

'Various deficiencies in applications include incorrect information, missing documents, and inadequate proof for investment, technical services, and financial details. These shortcomings hinder the approval process, requiring clarification and submission of additional supporting materials.'

In [ ]:
import re

text = "['<s> Develop a token classification model to accurately identify and classify named entities label:\n### Token:\nDescargarVista\n### label:\n1</s>']"

# Using regular expression to extract the label
label_match = re.search(r'### label:\n(\d+)', text)

if label_match:
    label = label_match.group(1)
    print("Extracted label:", label)
else:
    print("Label not found.")


Extracted label: 1


#### Many samples

In [ ]:
import re

for i, sample in enumerate(dataset['test']):
  inputs = tokenizer(
      [
          prompt.format(
          sample['tokens'], # insert comment
          "", # output - blank (instead of label) for generation
          )
      ],
      return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 20, use_cache = True)
  decoded_outputs = tokenizer.batch_decode(outputs)

  # extract "comment label" in response
  matches = re.findall(r'Token label:\n(\d)', decoded_outputs[0])
  predicted_label = int(matches[-1]) if matches else decoded_outputs

  print(f"Token {i}: {sample['tokens']}\nTrue label: {sample['label']}\nPredicted label:\n{predicted_label}\n")
  break

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Token 0: TheCartPress
True label: 1
Predicted label:
['<s> Develop a token classification model to accurately identify and classify named entities label:\n### Token:\nTheCartPress\n### label:\n1</s>']



In [ ]:
import re

def extract_label(text):
    label_match = re.search(r"label:\\\n(\d+)", str(text))
    if label_match:
        return label_match.group(1)
    else:
        return "Label not found."

predicted_labels = []  # Initialize an empty list to store predicted labels

for i, sample in enumerate(dataset['test']):
  inputs = tokenizer(
      [
          prompt.format(
          sample['tokens'], # insert comment
          "", # output - blank (instead of label) for generation
          )
      ],
      return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 20, use_cache = True)
  decoded_outputs = tokenizer.batch_decode(outputs)
  decoded_outputs = str(decoded_outputs[0])
  # print(decoded_outputs)
  # Using regular expression to extract the label
  label_match = re.search(r'### label:\n(\d+)', decoded_outputs)

  if label_match:
      label = label_match.group(1)
      predicted_labels.append(label)
      print("Extracted label:", label)
  else:
      print("Label not found.")

#   # extract "comment label" in response
#   matches = re.findall(r'Token label:\n(\d)', decoded_outputs[0])
#   predicted_label = int(matches[-1]) if matches else decoded_outputs
#   predicted_label = extract_label(predicted_label)
#   predicted_labels.append(predicted_label)  # Append the predicted label to the list

#   print(f"Token {i}: {sample['tokens']}\nTrue label: {sample['label']}\nPredicted label:\n{predicted_label}\n")
#   break
# # Now you have all the predicted labels in the predicted_labels list
# print("All predicted labels:", predicted_labels)



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Extracted label: 1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Extracted label: 4


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Extracted label: 14


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Extracted label: 4


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Extracted label: 22
Extracted label: 1


In [ ]:
predicted_labels

['1',
 '4',
 '22',
 '15',
 '1',
 '4',
 '23',
 '4',
 '4',
 '22',
 '22',
 '23',
 '1',
 '22',
 '5',
 '1',
 '2',
 '23',
 '23',
 '1',
 '23',
 '1',
 '23',
 '22',
 '23',
 '22',
 '4',
 '1',
 '22',
 '1',
 '4',
 '1',
 '1',
 '22',
 '23',
 '4',
 '22',
 '23',
 '22',
 '23',
 '4',
 '1',
 '4',
 '1',
 '22',
 '23',
 '4',
 '4',
 '22',
 '23',
 '5',
 '23',
 '1',
 '1',
 '15',
 '4',
 '4',
 '4',
 '22',
 '1',
 '1',
 '22',
 '1',
 '22',
 '4',
 '23',
 '22',
 '4',
 '1',
 '23',
 '5',
 '1',
 '5',
 '23',
 '23',
 '23',
 '1',
 '4',
 '23',
 '1',
 '4',
 '1',
 '23',
 '15',
 '14',
 '23',
 '22',
 '23',
 '1',
 '15',
 '23',
 '1',
 '22',
 '22',
 '4',
 '22',
 '4',
 '1',
 '5',
 '23',
 '23',
 '1',
 '2',
 '2',
 '23',
 '14',
 '22',
 '22',
 '4',
 '5',
 '5',
 '4',
 '22',
 '22',
 '22',
 '23',
 '1',
 '1',
 '22',
 '2',
 '4',
 '5',
 '22',
 '23',
 '1',
 '1',
 '23',
 '22',
 '4',
 '22',
 '22',
 '1',
 '4',
 '22',
 '4',
 '23',
 '10',
 '22',
 '23',
 '1',
 '1',
 '4',
 '23',
 '4',
 '23',
 '23',
 '22',
 '1',
 '23',
 '23',
 '22',
 '22',
 '22',
 '2

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Actual labels
actual = dataset['test']['label']

# Convert predicted labels to integers
predicted = [int(x) for x in predicted_labels]

# Calculate accuracy
accuracy = accuracy_score(actual, predicted)
print("Accuracy:", accuracy)

# Calculate precision
precision = precision_score(actual, predicted, average='weighted')  # Specify 'weighted' for multiclass tasks
print("Precision:", precision)

# Calculate recall
recall = recall_score(actual, predicted, average='weighted')  # Specify 'weighted' for multiclass tasks
print("Recall:", recall)

# Calculate F1 score
f1 = f1_score(actual, predicted, average='weighted')  # Specify 'weighted' for multiclass tasks
print("F1 Score:", f1)


Accuracy: 0.6012903225806452
Precision: 0.5512923451081723
Recall: 0.6012903225806452
F1 Score: 0.5528462316935755


In [ ]:
labels_dict = {'B-PL': 0, 'B-application': 1, 'B-cve id': 2, 'B-edition': 3, 'B-file': 4, 'B-function': 5, 'B-hardware': 6, 'B-language': 7, 'B-method': 8, 'B-os': 9, 'B-parameter': 10, 'B-relevant_term': 11, 'B-update': 12, 'B-vendor': 13, 'B-version': 14, 'I-application': 15, 'I-edition': 16, 'I-hardware': 17, 'I-os': 18, 'I-relevant_term': 19, 'I-update': 20, 'I-vendor': 21, 'I-version': 22, 'O': 23}

label_lists = list(labels_dict.keys())
value_list = list(labels_dict.values())

print("Label list:", label_lists)
print("Value list:", value_list)


Label list: ['B-PL', 'B-application', 'B-cve id', 'B-edition', 'B-file', 'B-function', 'B-hardware', 'B-language', 'B-method', 'B-os', 'B-parameter', 'B-relevant_term', 'B-update', 'B-vendor', 'B-version', 'I-application', 'I-edition', 'I-hardware', 'I-os', 'I-relevant_term', 'I-update', 'I-vendor', 'I-version', 'O']
Value list: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def class_metrics(y_true, y_pred, labels):
    # Initialize dictionaries to store metrics for each class
    accuracy = {}
    precision = {}
    recall = {}
    f1 = {}

    # Calculate confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    for i in range(len(labels)):
        label = labels[i]
        # Calculate metrics for each class
        tp = cm[i, i]
        fp = sum(cm[:, i]) - tp
        fn = sum(cm[i, :]) - tp
        tn = sum(sum(cm)) - tp - fp - fn

        # Calculate metrics
        accuracy[label] = (tp + tn) / (tp + tn + fp + fn)
        precision[label] = tp / (tp + fp) if (tp + fp) != 0 else 0
        recall[label] = tp / (tp + fn) if (tp + fn) != 0 else 0
        f1[label] = 2 * (precision[label] * recall[label]) / (precision[label] + recall[label]) if (precision[label] + recall[label]) != 0 else 0

    return accuracy, precision, recall, f1

# Example usage:

# True labels
y_true = dataset['test']['label']
# Predicted labels
y_pred =predicted
# Unique class labels
labels = value_list

accuracy, precision, recall, f1 = class_metrics(y_true, y_pred, labels)
labs = label_lists
# Print metrics for each class
for label, lab, val in zip(labels,labs, value_list):
  print(f"Class {lab}: Label: {val}")
  print(f"Accuracy: {accuracy[label]}")
  print(f"Precision: {precision[label]}")
  print(f"Recall: {recall[label]}")
  print(f"F1 Score: {f1[label]}")
  print()
